In [38]:
path = r'./'

In [39]:
import re
import time
import pickle
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import naive_bayes
from sklearn import metrics

In [54]:
def baggingClassify(df):
    return df.value_counts().index[0]

def getBaggingClassify(df):
    start = time.perf_counter()
    df_tmp = df
    df_tmp["predict"] = df[["predict_SVM","predict_Bayes","predict_XGB"]].apply(baggingClassify, axis = 1)
    end = time.perf_counter()
    timeCost = end - start
    
    print("\nBagging Classifiying Finish Transcript")
    print("-----------------------------------------")
    print("Time Cost (Seconds): ",round(timeCost,8))
    print("-----------------------------------------")

    return df_tmp


In [51]:
df_verify_SVM = pd.read_csv(path+"verified_data_SVM.csv", usecols = ["label","predict"])
df_verify_SVM.rename(columns = {"predict":'predict_SVM'}, inplace = True)

df_verify_Bayes = pd.read_csv(path+"verified_data_naiveBayes.csv", usecols = ["predict"])
df_verify_Bayes.rename(columns = {"predict":'predict_Bayes'}, inplace = True)

df_verify_XGB = pd.read_csv(path+"verified_data_XGB.csv", usecols = ["predict"])
df_verify_XGB.rename(columns = {"predict":'predict_XGB'}, inplace = True)

In [52]:
df_verify = df_verify_SVM.join(df_verify_Bayes)
df_verify = df_verify.join(df_verify_XGB)
display(df_verify)

,label,predict_SVM,predict_Bayes,predict_XGB
0,1,1,1,1.0
1,1,1,0,1.0
2,1,1,1,1.0
3,1,1,1,1.0
4,1,1,1,1.0
...,...,...,...,...
119983,0,1,1,0.0
119984,0,0,0,1.0
119985,0,0,0,0.0
119986,0,0,0,0.0


In [56]:
df_bagging_verified = getBaggingClassify(df_verify)


Bagging Classifiying Finish Transcript
-----------------------------------------
Time Cost (Seconds):  40.37512795
-----------------------------------------


In [57]:
display(df_bagging_verified)

,label,predict_SVM,predict_Bayes,predict_XGB,predict
0,1,1,1,1.0,1.0
1,1,1,0,1.0,1.0
2,1,1,1,1.0,1.0
3,1,1,1,1.0,1.0
4,1,1,1,1.0,1.0
...,...,...,...,...,...
119983,0,1,1,0.0,1.0
119984,0,0,0,1.0,0.0
119985,0,0,0,0.0,0.0
119986,0,0,0,0.0,0.0


In [58]:
y_veri = df_bagging_verified["label"]
y_pred = df_bagging_verified["predict"]
test_f1score = metrics.f1_score(y_veri, y_pred, average = "micro")
print(test_f1score)

0.8452428576190952


In [ ]:
# Bagging increases the f1score from 0.82 to 0.84

In [60]:
df_SVM = pd.read_csv(path+"predict_data_SVM.csv", usecols = ["timestamp","predict"])
df_SVM.rename(columns = {"predict":'predict_SVM'}, inplace = True)

df_Bayes = pd.read_csv(path+"predict_data_naiveBayes.csv", usecols = ["predict"])
df_Bayes.rename(columns = {"predict":'predict_Bayes'}, inplace = True)

df_XGB = pd.read_csv(path+"predict_data_XGB.csv", usecols = ["predict"])
df_XGB.rename(columns = {"predict":'predict_XGB'}, inplace = True)

In [61]:
df = df_SVM.join(df_Bayes)
df = df.join(df_XGB)
display(df)

,timestamp,predict_SVM,predict_Bayes,predict_XGB
0,2020-03-12 01:41:00,1,1,1.0
1,2020-03-12 01:41:00,0,0,1.0
2,2020-03-12 01:41:00,1,1,1.0
3,2020-03-12 01:41:00,1,1,1.0
4,2020-03-12 01:40:00,1,1,1.0
...,...,...,...,...
3112102,2020-02-13 16:58:00,1,0,1.0
3112103,2020-02-13 16:58:00,1,1,1.0
3112104,2020-02-13 16:58:00,1,1,1.0
3112105,2020-02-13 16:58:00,1,0,1.0


In [63]:
result = getBaggingClassify(df) # generally will take 20 mins to process
display(result)


Bagging Classifiying Finish Transcript
-----------------------------------------
Time Cost (Seconds):  1160.92687282
-----------------------------------------


,timestamp,predict_SVM,predict_Bayes,predict_XGB,predict
0,2020-03-12 01:41:00,1,1,1.0,1.0
1,2020-03-12 01:41:00,0,0,1.0,0.0
2,2020-03-12 01:41:00,1,1,1.0,1.0
3,2020-03-12 01:41:00,1,1,1.0,1.0
4,2020-03-12 01:40:00,1,1,1.0,1.0
...,...,...,...,...,...
3112102,2020-02-13 16:58:00,1,0,1.0,1.0
3112103,2020-02-13 16:58:00,1,1,1.0,1.0
3112104,2020-02-13 16:58:00,1,1,1.0,1.0
3112105,2020-02-13 16:58:00,1,0,1.0,1.0


In [64]:
bagging_result = pd.DataFrame()
bagging_result["predict"] = result["predict"]
bagging_result["timestamp"] = result["timestamp"]
bagging_result.to_csv("predict_data_Bagging.csv", index = False)

In [65]:
display(bagging_result)

,predict,timestamp
0,1.0,2020-03-12 01:41:00
1,0.0,2020-03-12 01:41:00
2,1.0,2020-03-12 01:41:00
3,1.0,2020-03-12 01:41:00
4,1.0,2020-03-12 01:40:00
...,...,...
3112102,1.0,2020-02-13 16:58:00
3112103,1.0,2020-02-13 16:58:00
3112104,1.0,2020-02-13 16:58:00
3112105,1.0,2020-02-13 16:58:00
